# 第3回
- LightGBM以外のモデルを学習してみる
- アンサンブルしてみる

In [ ]:
# 使用するライブラリ(numpy, pandas, matplotlib.pyplot)をインポートしましょう
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## データの読み込みと確認

In [ ]:
# 受領した各データ(train.csv, test.csv, sample_submission.csv)を読み込みましょう
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')
sample_submission = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')

In [ ]:
# trainデータの先頭5行を確認しましょう
train.head()

## データの加工
- ここはいくらでも手の加えようがあるので、自身で思いついた加工内容に変更してしまってOKです
- Cabin, PassengerIdから特徴量が作れないか検討してみましょう
- EDAから仮説を立てて、効きそうな特徴量を検討してみましょう
- 正解コードは簡易な書き方をしており、繰り返しが多いので、関数化できたらしてしまった方が楽かもしれません

In [ ]:
train.info()

In [ ]:
# 効きそうな特徴量作成
def getAgeBin(Age):
    if Age < 10:
        return 0
    else:
        return int(str(Age)[0])

def make_feature(df):
    # Cabinは、X/000/Xで構成されているため、それぞれ抜き出し
    df['Cabin_1'] = df['Cabin'].str.extract('(.+)/\d+/.+')
    df['Cabin_2'] = df['Cabin'].str.extract('.+/(\d+)/.+').astype(float)
    df['Cabin_3'] = df['Cabin'].str.extract('.+/\d+/(.+)')
    
    # nullを埋める
    # 最頻値で埋める
    for cols_mode in ['Cabin_1', 'Cabin_2', 'Cabin_3', 'HomePlanet', 'CryoSleep', 'Destination', 'VIP']:
        df[cols_mode] = df[cols_mode].fillna(df[cols_mode].mode()[0])
    # 0で埋める
    for cols_0 in ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
        df[cols_0] = df[cols_0].fillna(0)
    # 平均値で埋める
    for cols_mean in ['Age']:
        df[cols_mean] = df[cols_mean].fillna(df[cols_mean].mean())
    
    # RoomService, FoodCourt, ShoppingMall, Spa, VRDeckそれぞれについて、0と0以外で大きな差があったため0orNotの特徴量作成
    df['Room_0'] = np.where((df['RoomService']==0)|(df['RoomService'].isnull()), 1, 0)
    df['Food_0'] = np.where((df['FoodCourt']==0)|(df['FoodCourt'].isnull()), 1, 0)
    df['Shopping_0'] = np.where((df['ShoppingMall']==0)|(df['ShoppingMall'].isnull()), 1, 0)
    df['Spa_0'] = np.where((df['Spa']==0)|(df['Spa'].isnull()), 1, 0)
    df['VR_0'] = np.where((df['VRDeck']==0)|(df['VRDeck'].isnull()), 1, 0)
    
    # 上記のそれぞれについて、お金を使っていない場所が少ないほど乗客が少ないという仮説を立てたので、上記のフラグを合計する変数を作成
    df['0_Place_num'] = df['Room_0'] + df['Food_0'] + df['Shopping_0'] + df['Spa_0'] + df['VR_0']
    
    # 上記のそれぞれについて、合計額を特徴量として持たせる
    df['usedMoneySum'] = df['RoomService'].fillna(0) + df['FoodCourt'].fillna(0) + df['ShoppingMall'].fillna(0) + df['Spa'].fillna(0) + df['VRDeck'].fillna(0)
    
    # それぞれの場所で使った金額の割合を算出する
#     df['Room_rate'] = (df['RoomService'] / df['usedMoneySum']).fillna(0)
#     df['Food_rate'] = (df['FoodCourt'] / df['usedMoneySum']).fillna(0)
#     df['Shopping_rate'] = (df['ShoppingMall'] / df['usedMoneySum']).fillna(0)
#     df['Spa_rate'] = (df['Spa'] / df['usedMoneySum']).fillna(0)
#     df['VR_rate'] = (df['VRDeck'] / df['usedMoneySum']).fillna(0)
    
    # 年齢をビン分けする（x十代）
    df['AgeBin'] = df['Age'].apply(getAgeBin)
    
    return df

train = make_feature(train)
test = make_feature(test)

In [ ]:
# 分析用の加工
def make_df_for_analytics(df, categorical_cols):
    from sklearn.preprocessing import LabelEncoder
    # cabinを削除
    df = df.drop(['Cabin', 'PassengerId', 'Name'], axis=1)
    
    # カテゴリ変数(HomePlanet, Destination, Cabin_1, Cabin_3)の数値変換
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        
    # True_Falseの1, 0変換
    df['CryoSleep'] = df['CryoSleep'].astype(float)
    df['VIP'] = df['VIP'].astype(float)
    if 'Transported' in df.columns:
        df['Transported'] = df['Transported'].astype(int)
    
    return df
train = make_df_for_analytics(train, categorical_cols=['HomePlanet', 'Destination', 'Cabin_1', 'Cabin_3'])
test = make_df_for_analytics(test, categorical_cols=['HomePlanet', 'Destination', 'Cabin_1', 'Cabin_3'])

## バリデーションデータの作成

In [ ]:
# trainデータセットを、説明変数と目的変数に分割しましょう
X = train.drop('Transported', axis=1)
y = train['Transported']

In [ ]:
# train_test_split関数を用いて、データセットを訓練用と評価用に分割しましょう
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=0
)


## モデルの学習
- ここでは分析コンペでよく用いられるLightGBMを用います
- 他モデルを用いても良いですが、その場合以下のような加工が必要になります
     - nullを埋める
     - カテゴリ変数をone-hot encodingする(決定木系のモデルでない場合)
     - 標準化する(線形モデルやニューラルネットワークの場合)

In [ ]:
# 参考：https://blog.amedama.jp/entry/2018/05/01/081842
# 回答はscikit_learnインターフェースでない記法で書いております（パラメータチューニングやearly_stoppingなど、ググると日本語の記事が多くヒットする記法なため）
# lightgbmをインポートしましょう
import lightgbm as lgb

# データセットを生成しましょう
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

# lightGBMのパラメータを辞書型で定義しましょう（参考：https://qiita.com/nabenabe0928/items/6b9772131ba89da00354）
# lightGBMはパラメータ調整しなくても割といいスコアが出るので、「こんな書き方をするんだ」くらいに知っておけばOKです
lgb_params = {
    'objective': 'binary', 
    'metric': 'binary_logloss', 
    'verbosity': -1
}

# lightGBMを学習しましょう(early_stoppingをかけるようにコードを書いてみてください)
model = lgb.train(lgb_params, lgb_train, valid_sets=lgb_valid, num_boost_round=1000, early_stopping_rounds=10)


In [ ]:
# scikit-learn APIとそうでない記法では、出力の形が異なるため注意
# scikit-learn APIは予測のラベル(1/0)が、そうでない場合はラベルが1である予測確率が出力される
model.predict(X_valid)

In [ ]:
# catboostのモデルを作ってみる
import catboost as cb
# cat_featuresにカテゴリ変数を渡す場合、nullを埋める必要があるため注意
X_train_cat = X_train.copy()
X_valid_cat = X_valid.copy()
test_cat = test.copy()
cat_features = ['CryoSleep', 'VIP', 'HomePlanet', 'Destination', 'Room_0', 'Food_0', 'Shopping_0', 'Spa_0', 'VR_0', 'Cabin_1', 'Cabin_3', 'AgeBin']
for cat_col in cat_features:
    X_train_cat[cat_col] = X_train_cat[cat_col].fillna(0).astype(int)
    X_valid_cat[cat_col] = X_valid_cat[cat_col].fillna(0).astype(int)
    test_cat[cat_col] = test_cat[cat_col].fillna(0).astype(int)

import catboost as cb
cb_model = cb.CatBoostClassifier(eval_metric='Accuracy', early_stopping_rounds=10)

cb_model.fit(X_train_cat, y_train, eval_set=(X_valid_cat, y_valid), use_best_model=True, cat_features=cat_features)

In [ ]:
cb_model.predict(X_valid_cat)

In [ ]:
# XGBoostのモデルを作ってみる
import xgboost as xgb
xgb_train = xgb.DMatrix(X_train, y_train)
xgb_valid = xgb.DMatrix(X_valid, y_valid)

xgb_params = {
    'objective': 'binary:logistic', 
    'metric': 'logloss', 
}

xgb_model = xgb.train(xgb_params, xgb_train, evals=[(xgb_train, "train"), (xgb_valid, "valid")], num_boost_round=1000, early_stopping_rounds=10)

In [ ]:
xgb_model.predict(xgb_valid)

## モデルの評価

In [ ]:
# 評価用データから、予測を出力しましょう
y_pred = model.predict(X_valid)
# scikit-learn APIでない記法を用いている場合、予測確率0.5以上を1, そうでないものを0として予測を作成しましょう
y_pred = np.where(y_pred>=0.5, 1, 0)

# accuracy_score関数を用いて、スコアを計算しましょう
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_valid)

In [ ]:
# ↑セルと同様の評価を、catboostモデルに対しても行いましょう
y_pred_cat = cb_model.predict(X_valid_cat)
accuracy_score(y_pred_cat, y_valid)

In [ ]:
# ↑セルと同様の評価を、xgboostモデルに対しても行いましょう
y_pred_xgb = np.where(xgb_model.predict(xgb_valid)>=0.5, 1, 0)
accuracy_score(y_pred_xgb, y_valid)

## 提出用ファイルの作成

In [ ]:
# sample_submissionのTransported列を上書きする形で予測を作成します(他コンペも概ねその形を取ります)
# sample_submissionの先頭行を見て、形式を確認しましょう
sample_submission.head()

In [ ]:
# sample_submissionのTransported列を、モデルの予測に置き換えましょう
# 3モデルの出力を作成する
y_pred_lgb = model.predict(test)
y_pred_lgb = np.where(y_pred_lgb>=0.5, 1, 0)
# y_pred_cb = cb_model.predict(test_cat)
# y_pred_xgb = np.where(xgb_model.predict(xgb.DMatrix(test))>=0.5, 1, 0)

# アンサンブルしてみる（3モデルのうち、2モデル以上が1を示している場合1, そうでない場合は0を最終回答とする）
# y_pred = y_pred_lgb + y_pred_cb + y_pred_xgb
# y_pred = np.where(y_pred>=2, 1, 0)

# 予測は1/0でなされているため、bool型に変換するのを忘れないように
sample_submission['Transported'] = y_pred_lgb.astype(bool)
sample_submission.head()

In [ ]:
# csvとして出力（indexがファイル出力に含まれていると、提出形式に沿わなくなるため注意）
sample_submission.to_csv('submission.csv', index=False)